<a href="https://colab.research.google.com/github/vanitaa231/AICTE_CDD_VanitaAbnave/blob/master/Crop_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [16]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np


In [17]:
from google.colab import files
uploaded = files.upload()


Saving fruite_dataset.zip to fruite_dataset (1).zip


In [21]:
import os
os.listdir()


['.config',
 'fruite_dataset (1).zip',
 'dataset',
 'fruite_dataset.zip',
 'sample_data']

In [23]:
import zipfile

with zipfile.ZipFile("fruite_dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")


In [24]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    "dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    "dataset",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 7041 images belonging to 10 classes.
Found 1757 images belonging to 10 classes.


In [35]:
base_model = MobileNetV2(weights='imagenet',
                         include_top=False,
                         input_shape=(224,224,3))

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
# predictions = Dense(9, activation='softmax')(x)
num_classes = len(train_generator.class_indices)
predictions = Dense(num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [36]:
print(train_generator.class_indices)
print("Total classes:", len(train_generator.class_indices))


{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Grape___Black_rot': 4, 'Grape___Esca_(Black_Measles)': 5, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 6, 'Grape___healthy': 7, 'Strawberry___Leaf_scorch': 8, 'Strawberry___healthy': 9}
Total classes: 10


In [37]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

Epoch 1/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 145s 571ms/step - accuracy: 0.6850 - loss: 0.9809 - val_accuracy: 0.9619 - val_loss: 0.1263
Epoch 2/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 99s 448ms/step - accuracy: 0.9307 - loss: 0.2145 - val_accuracy: 0.9715 - val_loss: 0.0809
Epoch 3/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 99s 447ms/step - accuracy: 0.9488 - loss: 0.1512 - val_accuracy: 0.9715 - val_loss: 0.0886
Epoch 4/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 99s 449ms/step - accuracy: 0.9587 - loss: 0.1213 - val_accuracy: 0.9732 - val_loss: 0.0736
Epoch 5/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 100s 453ms/step - accuracy: 0.9634 - loss: 0.1061 - val_accuracy: 0.9789 - val_loss: 0.0698
Epoch 6/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 97s 441ms/step - accuracy: 0.9697 - loss: 0.0951 - val_accuracy: 0.9829 - val_loss: 0.0594
Epoch 7/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 98s 441ms/step - accuracy: 0.9619 - loss: 0.1096 - val_accuracy: 0.9806 - val_loss: 0.0499
Epoch 8/10
221/221 ━━━━━━━━━━━━━━━━━━━━ 99s 451ms/step - accuracy: 0.9721 - loss:

In [38]:
model.save("leaf_disease_model.h5")


In [40]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    confidence = np.max(prediction)

    class_labels = list(train_generator.class_indices.keys())
    result = class_labels[class_index]

    crop, disease = result.split("___")

    print("Crop:", crop)
    print("Disease:", disease)
    print("Confidence:", confidence)


In [41]:
from google.colab import files
files.download("leaf_disease_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
import tensorflow as tf
print(tf.__version__)


2.19.0


In [43]:
print(train_generator.class_indices)


{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Grape___Black_rot': 4, 'Grape___Esca_(Black_Measles)': 5, 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 6, 'Grape___healthy': 7, 'Strawberry___Leaf_scorch': 8, 'Strawberry___healthy': 9}


In [44]:
import json

with open("class_indices.json", "w") as f:
    json.dump(train_generator.class_indices, f)


In [45]:
from google.colab import files
files.download("class_indices.json")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>